In [1]:
import numpy as np

from openforcefield.topology import Molecule, Topology

from openff.system.stubs import ForceField
from openff.system.utils import get_test_file_path

In [11]:
# Load in a mainline OpenFF forcefield and construct a minimal ethanol topology
parsley = ForceField('openff-1.0.0.offxml')

top =  Topology.from_molecules(2 * [Molecule.from_smiles('CCO')])

In [12]:
# Use a monkey-patched function to parametrize the topology against a force field
sys_out = parsley.create_openff_system(top)

In [13]:
# Look at which ParameterHandler objects from the OpenFF toolkit 
# have been made into Potentialhandler objects
sys_out.handlers.keys()

dict_keys(['Bonds', 'Angles', 'ProperTorsions', 'vdW', 'Electrostatics'])

In [14]:
# Store the bond handler to inspect its contents
bonds = sys_out.handlers['Bonds']

In [15]:
# Look at some ~metadata
bonds.name, bonds.expression, bonds.independent_variables

('Bonds', '1/2 * k * (r - length) ** 2', {'r'})

In [16]:
# Return a mapping between atom indices and SMIRKS identifiers
bonds.slot_map

{(0, 1): '[#6X4:1]-[#6X4:2]',
 (0, 3): '[#6X4:1]-[#1:2]',
 (0, 4): '[#6X4:1]-[#1:2]',
 (0, 5): '[#6X4:1]-[#1:2]',
 (1, 2): '[#6:1]-[#8:2]',
 (1, 6): '[#6X4:1]-[#1:2]',
 (1, 7): '[#6X4:1]-[#1:2]',
 (2, 8): '[#8:1]-[#1:2]',
 (9, 10): '[#6X4:1]-[#6X4:2]',
 (9, 12): '[#6X4:1]-[#1:2]',
 (9, 13): '[#6X4:1]-[#1:2]',
 (9, 14): '[#6X4:1]-[#1:2]',
 (10, 11): '[#6:1]-[#8:2]',
 (10, 15): '[#6X4:1]-[#1:2]',
 (10, 16): '[#6X4:1]-[#1:2]',
 (11, 17): '[#8:1]-[#1:2]'}

In [17]:
# Return a mapping between SMIRKS identifiers and Potential objects;
# Note the de-duplication, resulting from a many-to-few mapping between
# bonds in the topology and unique parameters in the force field
vdw.potentials

{'[#6X4:1]-[#6X4:2]': Potential(parameters={'k': <Quantity(531.137374, 'kilocalorie / angstrom ** 2 / mole')>, 'length': <Quantity(1.5203759, 'angstrom')>}),
 '[#6X4:1]-[#1:2]': Potential(parameters={'k': <Quantity(758.093177, 'kilocalorie / angstrom ** 2 / mole')>, 'length': <Quantity(1.09288838, 'angstrom')>}),
 '[#6:1]-[#8:2]': Potential(parameters={'k': <Quantity(669.141517, 'kilocalorie / angstrom ** 2 / mole')>, 'length': <Quantity(1.41428792, 'angstrom')>}),
 '[#8:1]-[#1:2]': Potential(parameters={'k': <Quantity(1120.58324, 'kilocalorie / angstrom ** 2 / mole')>, 'length': <Quantity(0.970768759, 'angstrom')>})}

In [18]:
# Look at this contents of one of the Potential objects
bonds.potentials['[#6X4:1]-[#1:2]']

Potential(parameters={'k': <Quantity(758.093177, 'kilocalorie / angstrom ** 2 / mole')>, 'length': <Quantity(1.09288838, 'angstrom')>})

In [19]:
# Further, look at the particular value of one of its parameters
bonds.potentials['[#6X4:1]-[#1:2]'].parameters['k']

758.0931772913 <Unit('kilocalorie / angstrom ** 2 / mole')>

In [21]:
# Look up, from the highest-level object, this same data, using the
# SMIRKS pattern as a key connecting the topological data to the
# parametrized data
sys_out.handlers['Bonds'].potentials[
    sys_out.handlers['Bonds'].slot_map[(1, 7)]
].parameters['k']

758.0931772913 <Unit('kilocalorie / angstrom ** 2 / mole')>